<a href="https://colab.research.google.com/github/Marlburo/colab_nb/blob/Keras/colab_nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt


In [2]:
tf.config.threading.set_inter_op_parallelism_threads(
    0
)

#housing_test = pd.read_csv("/content/sample_data/california_housing_test.csv")
#housing_train = pd.read_csv("/content/sample_data/california_housing_train.csv")
mnist_test = pd.read_csv("/content/sample_data/mnist_test.csv")
mnist_train = pd.read_csv("/content/sample_data/mnist_train_small.csv")

In [3]:
#housing_test.describe()
#mnist_test
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name = 'predictions')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()#mnist_train, mnist_test

#preprocess the data
x_train = x_train.reshape(60000, 784).astype("float32")/255
x_test = x_test.reshape(10000, 784).astype("float32")/255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

#reserve 10000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

11493376/11490434 [==============================] - 0s 0us/step


In [5]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),
    #loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    #list of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()]
)

In [6]:
print("Fit model on training data")
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=15,
    #We pass some validation for
    #monitoring validation loss and metrics
    #at the end of each epoch
    validation_data=(x_val, y_val),
)

Fit model on training data
Epoch 1/15
782/782 [==============================] - 3s 3ms/step - loss: 0.3430 - sparse_categorical_accuracy: 0.9028 - val_loss: 0.1868 - val_sparse_categorical_accuracy: 0.9464
Epoch 2/15
782/782 [==============================] - 2s 3ms/step - loss: 0.1649 - sparse_categorical_accuracy: 0.9511 - val_loss: 0.1520 - val_sparse_categorical_accuracy: 0.9555
Epoch 3/15
782/782 [==============================] - 2s 3ms/step - loss: 0.1213 - sparse_categorical_accuracy: 0.9638 - val_loss: 0.1205 - val_sparse_categorical_accuracy: 0.9672
Epoch 4/15
782/782 [==============================] - 2s 3ms/step - loss: 0.0970 - sparse_categorical_accuracy: 0.9706 - val_loss: 0.1242 - val_sparse_categorical_accuracy: 0.9633
Epoch 5/15
782/782 [==============================] - 2s 3ms/step - loss: 0.0809 - sparse_categorical_accuracy: 0.9753 - val_loss: 0.1012 - val_sparse_categorical_accuracy: 0.9700
Epoch 6/15
782/782 [==============================] - 2s 3ms/step - loss:

In [7]:
history.history

{'loss': [0.342960387468338,
  0.1648763120174408,
  0.12128890305757523,
  0.09695176780223846,
  0.0809183344244957,
  0.06977476924657822,
  0.05966268107295036,
  0.052336376160383224,
  0.045596059411764145,
  0.04033646360039711,
  0.035589899867773056,
  0.031274549663066864,
  0.02796190418303013,
  0.02537720277905464,
  0.02185506746172905],
 'sparse_categorical_accuracy': [0.9028400182723999,
  0.9511399865150452,
  0.9638000130653381,
  0.9705600142478943,
  0.9753199815750122,
  0.9786400198936462,
  0.98198002576828,
  0.984279990196228,
  0.9865999817848206,
  0.9879199862480164,
  0.9893800020217896,
  0.9904000163078308,
  0.9914399981498718,
  0.9923200011253357,
  0.9937800168991089],
 'val_loss': [0.18680472671985626,
  0.15204842388629913,
  0.12049970775842667,
  0.124221570789814,
  0.10120357573032379,
  0.10533509403467178,
  0.09287604689598083,
  0.10147684067487717,
  0.15255491435527802,
  0.1047794446349144,
  0.11695603281259537,
  0.10391076654195786,
  

In [8]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print ("test loss, test acc", results)

#generate predictions (probabilities -- the output of the last layer
#on new data using predict
print ("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)

Evaluate on test data
79/79 [==============================] - 0s 2ms/step - loss: 0.1415 - sparse_categorical_accuracy: 0.9691
test loss, test acc [0.14148303866386414, 0.9690999984741211]
Generate predictions for 3 samples
predictions shape: (3, 10)


In [9]:
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [10]:
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)

In [11]:
def get_uncompiled_model():
  inputs = keras.Input(shape=(784,), name="digits")
  x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
  x = layers.Dense(64, activation="relu", name="dense_2")(x)
  outputs = layers.Dense(10, activation="softmax",name="predictions")(x)
  model = keras.Model(inputs=inputs, outputs=outputs)
  return model

def get_compiled_model():
  model = get_uncompiled_model()
  model.compile(
      optimizer="rmsprop",
      loss="sparse_categorical_crossentropy",
      metrics=["sparse_categorical_accuracy"],
  )
  return model

In [12]:
def custom_mean_squared_error(y_true, y_pred):
  return tf.math.reduce_mean(tf.square(y_true - y_pred))

model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.Adam(), loss=custom_mean_squared_error)

#We need to one-hot encode the labels to use MSE
y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=10)


Epoch 1/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0160
Epoch 2/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0080
Epoch 3/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0063
Epoch 4/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0050
Epoch 5/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0043
Epoch 6/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0036
Epoch 7/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0033
Epoch 8/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0029
Epoch 9/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0026
Epoch 10/10
782/782 [==============================] - 2s 2ms/step - loss: 0.0023


In [ ]:
class CustomMSE(keras.losses.Loss):
  def __init__(self, regularization_factor=0.1, name="custom_mse"):
    super().__init__(name=name)
    self.regularization_factor = regularization_factor

  def call(self, y_true, y_pred):
    mse=tf.math.reduce_mean(tf.square(y_true - y_pred))
    reg=tf.math.reduce_mean(tf.square(0.5 - y_pred))
    return mse + reg * self.regularization_factor

model = get_uncompiled_model()
model.compile(optimizer=keras.optimizers.Adam(), loss=CustomMSE())  

y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=20)


Epoch 1/20
782/782 [==============================] - 2s 2ms/step - loss: 0.0385
Epoch 2/20
782/782 [==============================] - 2s 2ms/step - loss: 0.0311
Epoch 3/20
782/782 [==============================] - 2s 2ms/step - loss: 0.0295
Epoch 4/20
782/782 [==============================] - 2s 2ms/step - loss: 0.0286
Epoch 5/20
782/782 [==============================] - 2s 2ms/step - loss: 0.0280
Epoch 6/20
782/782 [==============================] - 2s 2ms/step - loss: 0.0276
Epoch 7/20
782/782 [==============================] - 2s 2ms/step - loss: 0.0273
Epoch 8/20
782/782 [==============================] - 2s 2ms/step - loss: 0.0270
Epoch 9/20
782/782 [==============================] - 2s 2ms/step - loss: 0.0267
Epoch 10/20
782/782 [==============================] - 2s 2ms/step - loss: 0.0266
Epoch 11/20
782/782 [==============================] - 2s 2ms/step - loss: 0.0264
Epoch 12/20
782/782 [==============================] - 2s 2ms/step - loss: 0.0264


In [ ]:
class CategoricalTruePositives(keras.metrics.Metric):
  def __init__(self, name="categorical_true_positives", **kwargs):
    super(CategoricalTruePositives, self).__init__(name=name, **kwargs)
    self.true_positives = self.add_weight(name="ctp", initializer="zeros")

  def update_state(self, y_true, y_pred, sample_weight=None):
    y_pred = tf.reshape(tf.argmax(y_pred, axis=1), shape=(-1,1))
    values = tf.cast(y_true, "int32") == tf.cast(y_pred, "int32")
    values = tf.cast(values, "float32")
    if sample_weight is not None:
      sample_weight = tf.cast(sample_weight, "float32")
      values = tf.multiply(values, sample_weight)
    self.true_positives.assign_add(tf.reduce_sum(values))

  def result(self):
    return self.true_positives

  def reset_states(self):
    #The state of the metric will be reset at the start of each epoch.
    self.true_positives.assign(0.0)

model = get_uncompiled_model()
model.compile(
  optimizer = keras.optimizers.RMSprop(learning_rate=1e-3),
  loss=keras.losses.SparseCategoricalCrossentropy(),
  metrics=[CategoricalTruePositives()],  
)            
model.fit(x_train, y_train, batch_size=64, epochs=3)

In [ ]:
model.summary()